## None 을 반환하기보다는 예외를 발생시켜라

In [2]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
    
    
x, y = 1, 0
result = careful_divide(x, y)

if result is None:
    print("잘못된 입력")

잘못된 입력


제수가 0이 아닌 한 함수는 0을 반환한다. 그런데 함수가 반환한 결과를 if 문 등의 조건에서 평가할 때 0 값이 문제가 될 수 있다.

None인지 검사하는 대신, 실수로 빈 값을 False로 취급하는 검사를 실행할 수 있다.

In [3]:
x, y = 0, 5
result = careful_divide(x, y)

if not result:
    print("잘못된 입력")

잘못된 입력


False와 동등한 반환 값을 잘못 해석하는 경우는 None이 특별한 의미를 가지는 파이썬 코드에서 흔히 저지르는 실수다. 그래서 careful_divide 와 같은 함수에서 None 을 반환하는 오류를 야기하기 쉽다.

실수를 줄이는 방법은 두 가지이다.
- 반환 값을 2-튜플로 분리하는 것이다. 이 튜플의 첫 번째 부분은 연산이 성공인지 실패인지를 표시한다. 두 번째 부분은 계산에 성공한 경우 실제 결괏값을 저장한다.


In [ ]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return False, None
    
success, result = careful_divide(x, y)

if not success:
    print("잘못된 입력")

위 방법의 문제점은 호출하는 쪽에서 튜플의 첫 번째 부분(변수를 사용하지 않음을 밑줄로 표시하는 파이썬 표준 관례)을 쉽게 무시할 수 있다는 것이다. 이렇게 작성한 코드는 한눈에 보면 잘못됐는지 알아보기 어렵지만, None을 반환한 경우와 마찬가지로 실수할 가능성이 높아진다.

In [ ]:
_, result = careful_divide(x, y)

if not success:
    print("잘못된 입력")

더 나은 방법은 특별한 경우에 결코 None 을 반환하지 않는 것이다. 대신 Exception을 호출한 쪽으로 발생시켜서 호출자가 이를 처리하게 한다.

ZeroDivisionError -> ValueError 호출

In [6]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return ValueError('잘못된 입력')
    
x, y = 5, 2

try:
    result = careful_divide(x, y)
except ValueError:
    print("잘못된 입력")
else:
    print(f"결과는 {result:>0.1f}")

결과는 2.5


호출자는 더 이상 반환 값에 대한 조건문을 사용하지 않아도 된다. 대신에 반환 값이 항상 올바르다고 가정하고, try 문의 else 블록에서 이 값을 즉시 사용할 수 있다.

이 접근 방법을 확장해서 타입 애너테이션을 사용하는 코드에도 적용할 수 있다. 함수의 값이 항상 float이라고 지정할 수 있고, 그에 따라 None이 결코 반환되지 않음을 알릴 수 있다.

하지만 파이썬의 점진적 타입 지정에서는 함수의 인터페이스에 예외가 포함되는지 표현하는 방법이 의도적으로 제외됐다.

대신에 호출자가 어떤 Exception을 잡아내야 할지 결정할 때 문서를 참조할 것으로 예상하고, 발생시키는 예외를 문서에 명시해야 한다.

In [8]:
def careful_divide(a: float, b: float) -> float:
    """a를 b로 나눈다.
    
    Raises:
        ValueError: b가 0이어서 나눗셈을 할 수 없을 때
    """
    
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')

입력, 출력, 예외적인 동작이 모두 명확해졌고, 호출자가 이 함수를 잘못 처리할 가능성이 매우 낮아졌다.